<a href="https://colab.research.google.com/github/FakeAimanHafiy/fluffy-eureka/blob/Lab6/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="tic3151.png" style="height: 70px; width: 200px" align=left> 
<img src="logo-MMU.png" style="height: 70px; width: 200px" align=right>  



# **Lab 6: Constraint Satisfaction Problem (CSP)**

Towards the end of this lesson, you should be able to program in:
- understand constraint satisfaction problem

# Steps in CSP

General steps in solving CSP using ``Constraint`` package:

- import constraint
- define a variable as our problem
- add variables and their respective intervals to our problem
- add built-in/custom constraints to our problem
- fetch the solutions
- go through the solutions to find the ones we need

https://stackabuse.com/constraint-programming-with-python-constraint/

### Installing the necessary libraries

In [ ]:
#!pip install python-constraint
import constraint

## Sample 1

**x** is "constrained" to the values 1,2,3,4,5,6; 

**y** has to be less than 10 and their sum has to be greater than or equal to 10.  

In [ ]:
# Domain variable definition

# your answer here...
problem = constraint.Problem()

problem.addVariable('x',[1,2,3,4,5,6])
problem.addVariable('y',range(10))

In [ ]:
# Domain constraint definition

def our_constraint(x, y):  
    # your answer here...
    if x + y >= 10:
        return True

# your answer here...
problem.addConstraint(our_constraint,['x','y'])

In [ ]:
# Run solution

# your answer here...
solutions = problem.getSolutions()

# Prettier way to print and see all solutions
length = len(solutions)  
print("(x,y) ∈ {", end="")  
for index, solution in enumerate(solutions):  
    if index == length - 1:
        print("({},{})".format(solution['x'], solution['y']), end="")
    else:
        print("({},{}),".format(solution['x'], solution['y']), end="")
print("}")  

(x,y) ∈ {(6,9),(6,8),(6,7),(6,6),(6,5),(6,4),(5,9),(5,8),(5,7),(5,6),(5,5),(4,9),(4,8),(4,7),(4,6),(3,9),(3,8),(3,7),(2,9),(2,8),(1,9)}


## Sample 2

You recently got a job as a cashier. You're trying to convince your friend that it's hard work, there are just SO many ways to give someone their change back! Your "friend" shakes his head, obviously not believing you. He says "It can't be that bad. How many ways can there POSSIBLY be to give someone their change back, for like 60 cents?".

Your response is, of course, to sit and quickly write a program that would prove your point. You have a decent amount of pennies (1 cent), nickels (5 cents), dimes (10 cents) and quarters (25 cents), and a lot of kinda suspicious coins worth 3 cents each. Calculate in how many ways you can return change for 60 cents.  

**Note:** The order in which our result is printed isn't necessarily the same as the order in which we've added the variables. That is, if the result is (a,b,c,d,e) we don't know whether we have $a$ of 1 cent coins, $b$ of 3 cent coins, etc.

So we should explicitly print the variable and its value. One consequence of this is that we can't use the built-in ``.ExactSumConstraint()`` in its two-parameter form, ``ExactSumConstraint(50,[1,3,5,10,20])``.

The second parameter here is the "weight" of each variable (how many times it should be multiplied), and we have no guarantee which of our variables will have what weight.

In [ ]:
import constraint

problem = constraint.Problem()

# The maximum amount of each coin type can't be more than 60
# (coin_value*num_of_coints) <= 60

problem.addVariable("1 cent", range(61))  
problem.addVariable("3 cent", range(21))  
problem.addVariable("5 cent", range(13))  
problem.addVariable("10 cent", range(7))  
problem.addVariable("20 cent", range(4))

# your answer here...


# Where we explicitly give the order in which the weights should be allocated

# We could've used a custom constraint instead, BUT in this case the program will
# run slightly slower - this is because built-in functions are optimized and
# they find the solution more quickly
# def custom_constraint(a, b, c, d, e):
#     if a + 3*b + 5*c + 10*d + 20*e == 60:
#         return True
#     problem.addConstraint(o, ["1 cent", "3 cent", "5 cent","10 cent", "20 cent"])

def print_solutions(sols):  
    for s in sols:
        print("---")
        print("""
        1 cent: {0:d}
        3 cent: {1:d}
        5 cent: {2:d}
        10 cent: {3:d}
        20 cent: {4:d}""".format(s["1 cent"], s["3 cent"], s["5 cent"], s["10 cent"], s["20 cent"]))
        # If we wanted to we could check whether the sum was really 60
        # print("Total:", s["1 cent"] + s["3 cent"]*3 + s["5 cent"]*5 + s["10 cent"]*10 + s["20 cent"]*20)
        # print("---")

solutions = problem.getSolutions()  
#print_solutions(solutions)
print("Total number of ways: {}".format(len(solutions)))  

## Sample 3

You wish to pack chocolates for your mother. Luckily you work in a chocolate factory that has a lot of leftover chocolate. You have a few chocolate types at your disposal.

Your goal is to bring her the sweetest chocolate possible, that you can pack in your bag and sneak through security, and that wouldn't pass a certain net value for which you'd go to prison if you got caught.

Security most likely won't get suspicious if you bring less than 3kg. You can fit into **one decimeter** (1 $dm^3$) (a volume unit which is exactly equivalent to a litre) of chocolate in your bag. You won't go to jail if you steal less than $300 worth of chocolate.  

<img src="lab6_csp_fig.png" style="height: 200px; width: 800px" align=left> 

We'll first figure out how much of each chocolate we can have if we ONLY brought that type, so we can have the upper bound of our intervals. For example, for Chocolate A, based on weight we can bring at most 30 bars, based on value we can bring 37 at most, and based on volume we can bring 100.

The smallest of these numbers is 30, and that's the maximum number of Chocolate A we can bring. The same steps give us the maximum amount of the rest, B -> 44, C -> 75, D -> 100.

In [ ]:
import constraint

problem = constraint.Problem()

problem.addVariable('A', range(31))  #abcd is the name of the chocolate, the range is with
problem.addVariable('B', range(45))  #after calculations, we can see the max choco for A is 30, for b is 44, and so on.
problem.addVariable('C', range(76))  
problem.addVariable('D', range(101))

# We have 3kg = 3,000g available
def weight_constraint(a, b, c, d):  
    if (a*100 + b*45 + c*10 + d*25) <= 3000:
        return True

# We have 1dm^3 = 1,000cm^3 available
def volume_constraint(a, b, c, d):  
    if (a*8*2.5*0.5 + b*6*2*0.5 * c*2*2*0.5 + d*3*3*0.5) <= 1000:
        return True

# We can't exceed $300
def value_constraint(a, b, c, d):  
    if (a*8 + b*6.8 + c*4 + d*3) < 300:
        return True

# your answer here...

maximum_sweetness = 0  
solution_found = {}  
solutions = problem.getSolutions()

# Get the conditions with maximum sweetness

# your answer here...
problem.addConstraint(weight_constraint, "ABCD")
problem.addConstraint(volume_constraint, "ABCD")
problem.addConstraint(value_constraint, "ABCD")

maximum_sweetness=0
solution_found={}
solutions = problem.getSolutions()

#get conditions with max sweetness
for s in solutions:
    current_sweetness=s['A']*20 +s['B']*16 +s['C']*9 +s['D']*7
    if current_sweetness > maximum_sweetness:
        maximum_sweetness = current_sweetness
        solution_found = s

print("""  
The maximum sweetness we can bring is: {}  
We'll bring:  
{} A Chocolates,
{} B Chocolates,
{} C Chocolates,
{} D Chocolates
""".format(maximum_sweetness, solution_found['A'], solution_found['B'], solution_found['C'], solution_found['D']))

  
The maximum sweetness we can bring is: 730  
We'll bring:  
27 A Chocolates,
2 B Chocolates,
16 C Chocolates,
2 D Chocolates

